# Coursework 2 for Cardiac MR Image Segmentation (2020-2021)

The aim of this work is to create a neural network that segments cardiac MR images. The dataset is comprised of 200 96x96 greyscale cardiac MR images (CMRI) along the short axis. The goal is to segment them into four different categories: myocardium, left ventricle, right ventricle, and background. The data has been pulled from the Automated Caridiac Diagnosis Challenge (ACDC) (O. Bernard, A. Lalande, C. Zotti, F. Cervenansky, 2018). Below is an example of a CMRI (left) and the segmentation (called mask) (right).
![alt text](sample_dataset.png "Example of image and true segmentation")

CMRI allows for a non-invasive examination of the heart. Appliying image segmentation allows for calculation of important quantitative measures such as right and left ventricle volume, myocardial mass, and ejection fraction (Chen, Chen et al., 2020).

This is the ideal problem for a convolutional neural network - indeed as technology and deep learning have developed, the area of medical imagery has benefited greatly. Applying a CNN to pixel-wise classification allows for a much faster and less error-prone method of segmentation.

The dataset has been split into the following three subsets: 50% for training, 10% for validation, and the remaining 40% for testing. The goal of this work is to implement a known segmentation architecture to this dataset then perform hyper-parameter optimisation to maximise the dice score on the test set. The dice score between two generic masks A, B is defined as:
<img src="https://latex.codecogs.com/gif.latex?dice(A,&space;B)&space;=&space;\frac{2|A&space;\cap&space;B|}{|A|&plus;|B|}" title="dice(A, B) = \frac{2|A \cap B|}{|A|+|B|}" />
The dice score ranges between 0 and 1 with 0 being completely wrong and 1 being a perfect match. Since there are 4 classes to be segmented then even if the theoretical minimum is 0, by just classiying each pixel at random will achieve a dice score of 0.25. 

Below we define some two plotting functions: one that plots the image with the ground truth mask and another that plots the image with the ground truth mask and the predicted mask.

In [2]:
from matplotlib import pyplot as plt
def show_image_mask(img, mask, cmap='gray'): # visualisation
    fig = plt.figure(figsize=(5,5))
    plt.subplot(1, 2, 1)
    plt.imshow(img, cmap=cmap)
    plt.axis('off')
    plt.subplot(1, 2, 2)
    plt.imshow(mask, cmap=cmap)
    plt.axis('off')
    
#Additional function that plots the predicted segmentation along with the image and the ground truth
def show_image_mask_pred(img, mask, pred, cmap='gray'):
    fig = plt.figure(figsize=(5,5))
    plt.subplot(1, 3, 1)
    plt.imshow(img, cmap=cmap) #CMRI
    plt.axis('off')
    plt.subplot(1, 3, 2)
    plt.imshow(mask, cmap=cmap) #ground truth mask
    plt.axis('off')
    plt.subplot(1, 3, 3)
    plt.imshow(pred, cmap=cmap) # predicted mask
    plt.axis('off')
    plt.show()

Next, we define two classes of data loaders to store the training, validating, and test data in memory. Note that the validation data loader will be an instance of the `TrainDataset` class.

In [3]:
import torch
import torch.utils.data as data
import cv2
import os
from glob import glob
import numpy as np

class TrainDataset(data.Dataset):
    def __init__(self, root=''):
        super(TrainDataset, self).__init__()
        self.img_files = glob(os.path.join(root,'image','*.png'))
        self.mask_files = []
        for img_path in self.img_files:
            basename = os.path.basename(img_path)
            self.mask_files.append(os.path.join(root,'mask',basename[:-4]+'_mask.png'))
            
    def __getitem__(self, index):
            img_path = self.img_files[index]
            mask_path = self.mask_files[index]
            data = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
            label = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)
            return torch.from_numpy(data).float(), torch.from_numpy(label).float()

    def __len__(self):
        return len(self.img_files)

class TestDataset(data.Dataset):
    def __init__(self, root=''):
        super(TestDataset, self).__init__()
        self.img_files = glob(os.path.join(root,'image','*.png'))

    def __getitem__(self, index):
            img_path = self.img_files[index]
            data = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
            return torch.from_numpy(data).float()

    def __len__(self):
        return len(self.img_files)


In [4]:
import torch
import torch.utils.data as data
import cv2
import os
from glob import glob
import numpy as np

class TrainDataset(data.Dataset):
    def __init__(self, root=''):
        super(TrainDataset, self).__init__()
        self.img_files = glob(os.path.join(root,'image','*.png'))
        self.mask_files = []
        for img_path in self.img_files:
            basename = os.path.basename(img_path)
            self.mask_files.append(os.path.join(root,'mask',basename[:-4]+'_mask.png'))
            

    def __getitem__(self, index):
            img_path = self.img_files[index]
            mask_path = self.mask_files[index]
            data = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
            label = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)
            return torch.from_numpy(data).float(), torch.from_numpy(label).float()

    def __len__(self):
        return len(self.img_files)

class TestDataset(data.Dataset):
    def __init__(self, root=''):
        super(TestDataset, self).__init__()
        self.img_files = glob(os.path.join(root,'image','*.png'))

    def __getitem__(self, index):
            img_path = self.img_files[index]
            data = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
            return torch.from_numpy(data).float()

    def __len__(self):
        return len(self.img_files)


# Implementation
Now we define the network architecture. We opted to use the SegNet architecture (Badrinarayanan, Kendall and Cipolla, 2015) which is an encoder-decoder model for image segmentation. The idea is that the model applies a sequence of increasingly large number of convolutional filters and max-pooling layers to map the image to a low dimensional space (but with a large number of channels) and then to reverse the process by up-sampling and convolutional layers to return the image back to the same dimensionality of the input. Below is the architecture of SegNet taken from their paper.

![alt text](segnet_archi.png "Architecture of SegNet")

The first half of the network is the encoder and is topologically identical to the first 13 convolutional layers of the VGG16 architecture. Indeed, in the paper the authors apply transfer learning and initialise the encoder weights with the VGG16 weights. Then after the encoder there are 13 decoder layers each one coresponding to an encoder layer. At the end, instead of having a fully connected layer (such as the one found in VGG16), SegNet opts to map straight from the high-resolution feature map straight to a pixel-wise softmax classification. Each convolutional layer uses a 3x3 kernel with 1 padding. The number of filters can be seen in the code below. The other notable feature of SegNet is the connection between each max-pooling layer and its corresponding up-sampling. When the down-pooling occurs, the model stores the index of where the maximum value in the max-pooling occurs and uses those indices to up-sample. Below we define the model in `CNNSEG()`.

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNNSEG(nn.Module):
    def __init__(self):
        super(CNNSEG, self).__init__()
        # fill in the constructor for your model here
        #Encoder layers
        self.encoder_00 = nn.Sequential(*[nn.Conv2d(in_channels=1,out_channels=64,kernel_size=3,padding=1),
                                                nn.BatchNorm2d(64)])
        
        self.encoder_01 = nn.Sequential(*[nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3,padding=1),
                                                nn.BatchNorm2d(64)])
        
        self.encoder_10 = nn.Sequential(*[nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,padding=1),
                                                nn.BatchNorm2d(128)])
        
        self.encoder_11 = nn.Sequential(*[nn.Conv2d(in_channels=128,out_channels=128,kernel_size=3,padding=1),
                                                nn.BatchNorm2d(128)])
        
        self.encoder_20 = nn.Sequential(*[nn.Conv2d(in_channels=128,out_channels=256,kernel_size=3,padding=1),
                                                nn.BatchNorm2d(256)])
        
        self.encoder_21 = nn.Sequential(*[nn.Conv2d(in_channels=256,out_channels=256,kernel_size=3,padding=1),
                                                nn.BatchNorm2d(256)])
        
        self.encoder_22 = nn.Sequential(*[nn.Conv2d(in_channels=256,out_channels=256,kernel_size=3,padding=1),
                                                nn.BatchNorm2d(256)])
        
        self.encoder_30 = nn.Sequential(*[nn.Conv2d(in_channels=256,out_channels=512,kernel_size=3,padding=1),
                                                nn.BatchNorm2d(512)])
        
        self.encoder_31 = nn.Sequential(*[nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,padding=1),
                                                nn.BatchNorm2d(512)])
        
        self.encoder_32 = nn.Sequential(*[nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,padding=1),
                                                nn.BatchNorm2d(512)])
        
        self.encoder_40 = nn.Sequential(*[nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,padding=1),
                                                nn.BatchNorm2d(512)])
        
        self.encoder_41 = nn.Sequential(*[nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,padding=1),
                                                nn.BatchNorm2d(512)])
        
        self.encoder_42 = nn.Sequential(*[nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,padding=1),
                                                nn.BatchNorm2d(512)])
        
        #Decoder layers
        
        
        self.decoder_42 = nn.Sequential(*[nn.ConvTranspose2d(in_channels=512,out_channels=512,kernel_size=3,padding=1),
                                                nn.BatchNorm2d(512)])
        
        self.decoder_41 = nn.Sequential(*[nn.ConvTranspose2d(in_channels=512,out_channels=512,kernel_size=3,padding=1),
                                                nn.BatchNorm2d(512)])
        
        self.decoder_40 = nn.Sequential(*[nn.ConvTranspose2d(in_channels=512,out_channels=512,kernel_size=3,padding=1),
                                                nn.BatchNorm2d(512)])
        
        self.decoder_32 = nn.Sequential(*[nn.ConvTranspose2d(in_channels=512,out_channels=512,kernel_size=3,padding=1),
                                                nn.BatchNorm2d(512)])
        
        self.decoder_31 = nn.Sequential(*[nn.ConvTranspose2d(in_channels=512,out_channels=512,kernel_size=3,padding=1),
                                                nn.BatchNorm2d(512)])
        
        self.decoder_30 = nn.Sequential(*[nn.ConvTranspose2d(in_channels=512,out_channels=256,kernel_size=3,padding=1),
                                                nn.BatchNorm2d(256)])
        
        self.decoder_22 = nn.Sequential(*[nn.ConvTranspose2d(in_channels=256,out_channels=256,kernel_size=3,padding=1),
                                                nn.BatchNorm2d(256)])
        
        self.decoder_21 = nn.Sequential(*[nn.ConvTranspose2d(in_channels=256,out_channels=256,kernel_size=3,padding=1),
                                                nn.BatchNorm2d(256)])
        
        self.decoder_20 = nn.Sequential(*[nn.ConvTranspose2d(in_channels=256,out_channels=128,kernel_size=3,padding=1),
                                                nn.BatchNorm2d(128)])
        
        self.decoder_11 = nn.Sequential(*[nn.ConvTranspose2d(in_channels=128,out_channels=128,kernel_size=3,padding=1),
                                                nn.BatchNorm2d(128)])
        
        self.decoder_10 = nn.Sequential(*[nn.ConvTranspose2d(in_channels=128,out_channels=64,kernel_size=3,padding=1),
                                                nn.BatchNorm2d(64)])
        
        self.decoder_01 = nn.Sequential(*[nn.ConvTranspose2d(in_channels=64,out_channels=64,kernel_size=3,padding=1),
                                                nn.BatchNorm2d(64)])
        
        self.decoder_00 = nn.Sequential(*[nn.ConvTranspose2d(in_channels=64,out_channels=4,kernel_size=3,padding=1)])
        
        
    def forward(self, x):
        # Encoder

        # Encoder Stage - 1
        dim_0 = x.size()
        x_00 = F.relu(self.encoder_00(x))
        x_01 = F.relu(self.encoder_01(x_00))
        x_0, indices_0 = F.max_pool2d(x_01, kernel_size=2, stride=2, return_indices=True)

        # Encoder Stage - 2
        dim_1 = x_0.size()
        x_10 = F.relu(self.encoder_10(x_0))
        x_11 = F.relu(self.encoder_11(x_10))
        x_1, indices_1 = F.max_pool2d(x_11, kernel_size=2, stride=2, return_indices=True)

        # Encoder Stage - 3
        dim_2 = x_1.size()
        x_20 = F.relu(self.encoder_20(x_1))
        x_21 = F.relu(self.encoder_21(x_20))
        x_22 = F.relu(self.encoder_22(x_21))
        x_2, indices_2 = F.max_pool2d(x_22, kernel_size=2, stride=2, return_indices=True)
        
        # Encoder Stage - 4
        dim_3 = x_2.size()
        x_30 = F.relu(self.encoder_30(x_2))
        x_31 = F.relu(self.encoder_31(x_30))
        x_32 = F.relu(self.encoder_32(x_31))
        x_3, indices_3 = F.max_pool2d(x_32, kernel_size=2, stride=2, return_indices=True)
        
        # Encoder Stage - 5
        dim_4 = x_3.size()
        x_40 = F.relu(self.encoder_40(x_3))
        x_41 = F.relu(self.encoder_41(x_40))
        x_42 = F.relu(self.encoder_42(x_41))
        x_4, indices_4 = F.max_pool2d(x_42, kernel_size=2, stride=2, return_indices=True)

        # Decoder

        dim_d = x_4.size()

        # Decoder Stage - 5
        x_4d = F.max_unpool2d(x_4, indices_4, kernel_size=2, stride=2, output_size=dim_4)
        x_42d = F.relu(self.decoder_42(x_4d))
        x_41d = F.relu(self.decoder_41(x_42d))
        x_40d = F.relu(self.decoder_40(x_41d))
        
        # Decoder Stage - 4
        x_3d = F.max_unpool2d(x_3, indices_3, kernel_size=2, stride=2, output_size=dim_3)
        x_32d = F.relu(self.decoder_32(x_3d))
        x_31d = F.relu(self.decoder_31(x_32d))
        x_30d = F.relu(self.decoder_30(x_31d))
        
        # Decoder Stage - 3
        x_2d = F.max_unpool2d(x_30d, indices_2, kernel_size=2, stride=2, output_size=dim_2)
        x_22d = F.relu(self.decoder_22(x_2d))
        x_21d = F.relu(self.decoder_21(x_22d))
        x_20d = F.relu(self.decoder_20(x_21d))

        # Decoder Stage - 2
        x_1d = F.max_unpool2d(x_20d, indices_1, kernel_size=2, stride=2, output_size=dim_1)
        x_11d = F.relu(self.decoder_11(x_1d))
        x_10d = F.relu(self.decoder_10(x_11d))

        # Decoder Stage - 1
        x_0d = F.max_unpool2d(x_10d, indices_0, kernel_size=2, stride=2, output_size=dim_0)
        x_01d = F.relu(self.decoder_01(x_0d))
        x_00d = self.decoder_00(x_01d)

        x_softmax = F.softmax(x_00d, dim=1)

        return x_softmax

model = CNNSEG()

We can see the summary of the model including the shape of the tensor at each step and the number of trainable parameters. Due to the fact that the network is fully convolutional and the use of max-pooling indices in up-sampling allows the model to have relatively less trainable parameters. 

In [6]:
from torchsummary import summary
summary(model, (1, 96, 96))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 96, 96]             640
       BatchNorm2d-2           [-1, 64, 96, 96]             128
            Conv2d-3           [-1, 64, 96, 96]          36,928
       BatchNorm2d-4           [-1, 64, 96, 96]             128
            Conv2d-5          [-1, 128, 48, 48]          73,856
       BatchNorm2d-6          [-1, 128, 48, 48]             256
            Conv2d-7          [-1, 128, 48, 48]         147,584
       BatchNorm2d-8          [-1, 128, 48, 48]             256
            Conv2d-9          [-1, 256, 24, 24]         295,168
      BatchNorm2d-10          [-1, 256, 24, 24]             512
           Conv2d-11          [-1, 256, 24, 24]         590,080
      BatchNorm2d-12          [-1, 256, 24, 24]             512
           Conv2d-13          [-1, 256, 24, 24]         590,080
      BatchNorm2d-14          [-1, 256,

In accordance with the SegNet paper, the loss function was chosen to be categorical cross-entropy. This is the natural choice for classification problems and is perfect for our case as we are performing pixel-wise classification. Unlike in the paper, we opted to use the Adam optimiser as it is currently one of the most popular optimisers. We allow pytorch to use GPU acceleration if it is available. Lastly, we initialise the learning rate at 0.001 as well as training and validation loss lists to store the training history.

In [7]:
#Set the learning rate
LEARNING_RATE = 0.001

#Initialise cross entropy loss and adam optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                                     lr=LEARNING_RATE)

#if gpu is available then it sends it to gpu else it uses cpu 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
if torch.cuda.is_available():
    model.cuda()

#Initialise list to store loss over training
train_loss = list()
val_loss = list()

Below we define some helper functions:
- `categorical_dice` which computes the categorical dice between two masks for a specific class.
- `forward` which preprocesses the image tensor by making it the right dimension and dividing each element component-wise by 255 to make the tensor lie inside the hypercube (granted of very high dimension). The gradients get set to zero in the optimiser and we push the mini-batch forwards through the network and return the softmax output of the model as a tensor of shape Bx4x96x96 where B is the batch size, 4 corresponds to the four categories and 96x96 is the image pixels.

In [8]:
#categorical dice score between mask1 and mask2 along label_class
def categorical_dice(mask1, mask2, label_class=1):
    mask1_pos = (mask1 == label_class).numpy()
    mask2_pos = (mask2 == label_class).numpy()
    dice = 2 * np.sum(mask1_pos * mask2_pos) / (np.sum(mask1_pos) + np.sum(mask2_pos))
    return dice

#Packages proprocessing and forward pass
def forward(img):
    #scales image so that each pixel lies between 0 and 1
    img_scale = img/255.
    
    #reshapes the tensor 
    img_scale = img_scale.unsqueeze(1)
    
    #resets optimiser 
    optimizer.zero_grad()
    
    #pushes processed image through network
    seg_soft = model(img_scale)
    
    return seg_soft

In [11]:
from torch.utils.data import DataLoader
data_path = './data/train'
val_path = './data/val'
num_workers = 8
batch_size = 10
train_set = TrainDataset(data_path)
val_set = TrainDataset(val_path)

training_data_loader = DataLoader(dataset=train_set, num_workers=num_workers, batch_size=batch_size, shuffle=True)
validating_data_loader = DataLoader(dataset=val_set, num_workers=num_workers, batch_size=20, shuffle=True)


data_loader = {'train':training_data_loader,
              'validate':validating_data_loader}

EPOCHS = 0
Max_Epoch = 100

In [ ]:
tol = 1e-5 #Tolerance perameter
print('Loss function ', loss_fn )
print('optimizer ', optimizer)
print('Mini Batch size set to ', batch_size)
print(device)

# Fetch images and labels. 
exit = False
while not exit:
    print('-' * 30)
    print('EPOCHS = ',EPOCHS)

    model.train(True)
    batch_loss = 0.0

    for index_train, sample in enumerate(data_loader['train']):
        
        img, mask = sample
        
        img, mask = img.to(device), mask.to(device)
        
        soft_mask = forward(img)
        
        loss = loss_fn(soft_mask, mask.long())

        loss.backward()
        optimizer.step()

        batch_loss += loss.item()

    train_loss.append(batch_loss / (index_train + 1))
    print("Train loss: ", train_loss[-1])
    
    model.train(False)

    batch_loss = 0.0
    for index_val, sample in enumerate(data_loader['validate']):
        img, mask = sample
        img, mask = img.to(device), mask.to(device)
        soft_mask = forward(img)
       
        loss = loss_fn(soft_mask, mask.long())

        batch_loss += loss.item()

    val_loss.append(batch_loss / (index_val + 1))
    print("Validation loss: ", val_loss[-1])
    
    if EPOCHS > 2:
        if (np.absolute(val_loss[-1] - val_loss[-2]) < tol) or (EPOCHS == Max_Epoch ):
            exit = True
    EPOCHS += 1
    
if (EPOCHS-1) == Max_Epoch:
    print('Failed to converge')
else:
    print('Converged in ',EPOCHS-1, ' epochs')


# References
Badrinarayanan, V., Kendall, A. and Cipolla, R., 2015. Segnet: A Deep Convolutional Encoder-Decoder Architecture For Image Segmentation. [online] arXiv.org. Available at: <https://arxiv.org/abs/1511.00561v3> [Accessed 14 December 2020].

O. Bernard, A. Lalande, C. Zotti, F. Cervenansky, et al.
"Deep Learning Techniques for Automatic MRI Cardiac Multi-structures Segmentation and
Diagnosis: Is the Problem Solved ?" in IEEE Transactions on Medical Imaging,
vol. 37, no. 11, pp. 2514-2525, Nov. 2018
doi: 10.1109/TMI.2018.2837502

Chen, Chen et al. “Deep Learning for Cardiac Image Segmentation: A Review.” Frontiers in cardiovascular medicine vol. 7 25. 5 Mar. 2020, doi:10.3389/fcvm.2020.00025